<a id='Q0'></a>
<center> <h1>Exploring German elections</h1> </center>
<p style="margin-bottom:1cm;"></p>
<center><h4>2025</h4></center>
<p style="margin-bottom:1cm;"></p>

<div style="background:#EEEDF5;border-top:0.1cm solid #EF475B;border-bottom:0.1cm solid #EF475B;">
    <div style="margin-left: 0.5cm;margin-top: 0.5cm;margin-bottom: 0.5cm;color:#303030">
        <p><strong>Goal:</strong> Short description of the aim of the notebook</p>
        <strong> Outline:</strong>
        <a id='P0' name="P0"></a>
        <ol>
            <li> <a style="color:#303030" href='#I'>Introduction </a> </li>
            <li> <a style="color:#303030" href='#SU'>Set up</a></li>
            <li> <a style="color:#303030" href='#P1'>Part 1 - exploring the data</a></li>
            <li> <a style="color:#303030" href='#P2'>Part 2</a></li>
            <li> <a style="color:#303030" href='#P3'>Part 3</a></li>
            <li> <a style="color:#303030" href='#CL'>Conclusion</a></li>
        </ol>
        <strong>Keywords:</strong> data cleaning, NLP, ... list of keywords.
    </div>
</div>
</nav>

<a id='I' name="I"></a>
## [Introduction](#P0)

Here you could write an introduction to the topics. Everything that is written here will appear in the live coding template notebook

<a id='SU' name="SU"></a>
## [Set up](#P0)

### Packages

In [ ]:
# General
import pandas as pd

# Visualization
import plotly.express as px

<a id='P1'></a>
## [Exploring the data](#P0)


### Elections dataset exploration

In [ ]:
elections = pd.read_csv("../data/federal_muni_harm_25.csv")

In [ ]:
print(elections.shape)
print(elections.columns.tolist())
print(elections["election_year"].unique())
print(elections["state"].unique())
print(elections["county"].nunique())
print(type(elections["state"][0]))

In [ ]:
print(type(elections["county"][0]))

# transform county into str and adapt it for the map
elections["county"] = elections["county"].apply(lambda x: f"0{x}" if len(str(x))==4 else str(x))
elections["county"]

In [ ]:
# save state code as str so that it can be used with the geodata

elections["state_code"] = elections["state"].astype(str)
print(type(elections["state_code"][0]))
elections["state_code"] = elections["state_code"].apply(lambda x: f"0{x}" if len(x)<2 else x)
elections["state_code"].head()

In [ ]:
# what does htis do?
# percentage of NAs in area_cw -> 100!
elections["area_cw"].isna().sum()/len(elections)

In [ ]:
# to see all columns
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
elections.head()

In [ ]:
# replace all NaN with 0 for the parties columns
elections.loc[:, "cdu":"zentrum"] = elections.loc[:, "cdu":"zentrum"].fillna(0)
elections.head()

In [ ]:
# sum all other parties after afd as others 
elections["other_parties"] = elections.loc[:, "npd":"werteunion"].sum(axis=1)
elections.head()

In [ ]:
# check if the sum of all parties is 1 (that is 100%), which means i included all parties
elections["votes_sum"] = elections.loc[:, "cdu":"werteunion"].sum(axis=1)
elections.head()

In [ ]:
# drop the other parties from the dataframe (since )
parties_to_drop = elections.loc[:, "npd":"werteunion"]
elections_short = elections.drop(columns=parties_to_drop)
elections_short.head()

In [ ]:
# drop some more unused columns
cols_to_drop =["ags", "state", "eligible_voters_orig", "number_voters_orig", "area_cw"]
elections_short = elections_short.drop(columns=cols_to_drop)
elections_short.head()

In [ ]:
elections_short

In [ ]:
# group by district and keep the average percentage of votes for each party
# will average everything, which is ok, because on percentages is better to take average
elects_grouped = elections_short.groupby(["election_year", "state_code", "county"]).mean(numeric_only=True).reset_index()       # reset_index() so that the columns don't transform to index
elects_grouped

In [ ]:
parties_cols = list(elects_grouped.loc[:, "cdu":"afd"]) + ["other_parties"]

In [ ]:
# recalculate the percentage of votes, to see if it's 100% -> it is 
elects_grouped["votes_sum_recalc"] = elects_grouped[parties_cols].sum(axis=1)
elects_grouped.head()

In [ ]:
# make a column with the winner of elections
elects_grouped["winner"] = elects_grouped[parties_cols].idxmax(axis=1)          # takes the max value from the parties_cols, but instead of saving the value, it saves the column name
elects_grouped.head()

In [ ]:
# reverting the option to see all columns
pd.reset_option('display.max_columns')
pd.reset_option('display.width')

In [ ]:
print(elections.columns.tolist())

### Economy dataset exploration

In [ ]:
# check encoding
import chardet

# Read a sample of the file
with open("../data/income.csv", "rb") as f:
    result = chardet.detect(f.read(100000))  # read first 100KB
print(result)

In [ ]:
# standard csv reading -> not so nice
income = pd.read_csv("../data/income.csv", encoding=result["encoding"], sep=";", skiprows=5, header=[0,1], index_col=[0,1])
income.head()

In [ ]:
# improved csv reading
# NB when giving names, it attributes names from right to left, every remaining column on the left is incorporated in the index
# skipfooter=4 → skips the last 4 rows of the file
# engine="python" → required because the default C engine doesn’t support skipfooter

income = pd.read_csv("../data/income.csv", encoding=result["encoding"], sep=";", skiprows=7, names=["year", "code", "region", "anzahl_steuerpflichtige", "gesamtbetrag", "steuer"], skipfooter=4, engine="python")
income.head()

In [ ]:
income.tail()

In [ ]:
# getting the codes for the lands (since not in alphabetical order)

land_codes = []
for i in range(16):
    if i<9:
        land_codes.append(f"0{i+1}")
    else:
        land_codes.append(f"{i+1}")

land_codes_dict ={}
income_lands = income[income["code"].isin(land_codes)].reset_index(drop=True)
income_lands_2021 = income_lands[income_lands["year"]==2021].reset_index(drop=True)
income_lands_2021.index
for i, row in income_lands_2021.iterrows():
    land_codes_dict[row["code"]] = row["region"].strip()        # they have some empty space left 
    # print(row["region"])

land_codes_dict

In [ ]:
print(type(income["gesamtbetrag"][0]))
print(type(income["steuer"][0]))
print(type(income["anzahl_steuerpflichtige"][0]))

In [ ]:
income["gesamtbetrag"] = pd.to_numeric(income["gesamtbetrag"], errors="coerce")         # invalid parsing becomes NaN
income["steuer"] = pd.to_numeric(income["steuer"], errors="coerce")         # invalid parsing becomes NaN
income["anzahl_steuerpflichtige"] = pd.to_numeric(income["anzahl_steuerpflichtige"], errors="coerce")         # invalid parsing becomes NaN

<a id='P2' name="P2"></a>
## [Adding the geojson](#P0)

In [ ]:
import json
geojson = json.load(open("../data/georef-germany-kreis.geojson"))

In [ ]:
geojson["features"][0]["properties"]

In [ ]:
income[income["code"]=="08125"]

In [ ]:
elections[elections["county"]=="08125"]

<a id='P3' name="P3"></a>
## [Plotting some test maps ](#P0)

### Elections

#### Original elections dataframe

In [ ]:
elections.head()

In [ ]:
sorted_elections = elections.sort_values("election_year").reset_index(drop=True)

In [ ]:
sorted_elections["percentage_voters"] = (sorted_elections["number_voters"]/sorted_elections["eligible_voters"])*100

In [ ]:
# fig = px.choropleth_map(
#     sorted_elections[sorted_elections["election_year"] == 2025],
#     geojson=geojson,
#     locations="county",
#     featureidkey="properties.krs_code",
#     color="percentage_voters",
#     hover_name="county",
#     zoom=5,
#     title="Percentage of people who voted",
#     labels={'percentage_voters': 'People voting (%)'},
#     width=900, height=650,
#     range_color=(0, 100)
# )
# fig.update_layout(
#     map_center={"lat": 51, "lon": 10},
#     autosize=False,
#     margin={"r": 0, "t": 0, "l": 0, "b": 0}
# )
# fig.write_html("percentage_voters_elections_2025.html")
# fig.show()

In [ ]:
# animated_fig = px.choropleth_map(
#     sorted_elections,
#     geojson=geojson,
#     locations="county",
#     featureidkey="properties.krs_code",
#     color="percentage_voters",
#     animation_frame="election_year",
#     hover_name="county",
#     zoom=4,
#     title="Percentage of people who voted",
#     labels={'percentage_voters': 'People voting (%)'},
#     width=900, height=650,
#     range_color=(0, 100)
# )
# animated_fig.update_layout(
#     map_center={"lat": 51, "lon": 10},
#     autosize=False,
#     margin={"r": 0, "t": 0, "l": 0, "b": 0}
# )
# animated_fig.write_html("percentage_voters_elections_over_time.html")
# animated_fig.show()

In [ ]:
# animated_fig = px.choropleth_map(
#     sorted_elections,
#     geojson=geojson,
#     locations="county",
#     featureidkey="properties.krs_code",
#     color="far_right",
#     animation_frame="election_year",
#     hover_name="county",
#     zoom=5,
#     title="Percentage of people voting far right",
#     labels={'far_right': 'People voting far right (%)'},
#     width=900, height=650,
#     range_color=(0, 1)
# )
# animated_fig.update_layout(
#     map_center={"lat": 51, "lon": 10},
#     autosize=False,
#     margin={"r": 0, "t": 0, "l": 0, "b": 0}
# )
# animated_fig.write_html("percentage_voters_far_right_over_time.html")
# animated_fig.show()

In [ ]:
# animated_fig = px.choropleth_map(
#     sorted_elections,
#     geojson=geojson,
#     locations="county",
#     featureidkey="properties.krs_code",
#     color="far_left",
#     animation_frame="election_year",
#     hover_name="county",
#     zoom=5,
#     title="Percentage of people voting far left",
#     labels={'far_left': 'People voting far left (%)'},
#     width=900, height=650,
#     range_color=(0, 0.1)
# )
# animated_fig.update_layout(
#     map_center={"lat": 51, "lon": 10},
#     autosize=False,
#     margin={"r": 0, "t": 0, "l": 0, "b": 0}
# )
# animated_fig.write_html("percentage_voters_far_left_over_time.html")
# animated_fig.show()

In [ ]:
# for col in elections.columns:
#     print(col)

#### Shortened elections dataframe

In [ ]:
elects_grouped.head()

In [ ]:
sorted_elects = elects_grouped.sort_values("election_year").reset_index(drop=True)
sorted_elects.head()

In [ ]:
sorted_elects[sorted_elects["election_year"]==2025]

Parties colors
https://gist.github.com/Fischaela/0cf760f17672e3eb399193e48d7c6104

In [ ]:
fig = px.choropleth_map(
    sorted_elects[sorted_elects["election_year"] == 2025],
    geojson=geojson,
    locations="county",
    featureidkey="properties.krs_code",
    color="winner",
    hover_name="winner",
    zoom=5,
    title="elected party per district",
    labels={'winner': 'Party'},
    color_discrete_map={
        'cdu':'#003B6F',
        'spd':'#A6006B',
        'gruene':'#1AA037',
        'fdp':'#FFEF00',
        'linke_pds':'#E3000F',
        'afd':'#0489DB'
    },
    width=900, height=650,
)
fig.update_layout(
    map_center={"lat": 51, "lon": 10},
    autosize=False,
    margin={"r": 0, "t": 0, "l": 0, "b": 0}
)
fig.show()

### Income

In [ ]:
income.head()

In [ ]:
# add a percentage of tax paid
income["tax_perc"] = (income["steuer"]/income["gesamtbetrag"])*100
income["income_per_capita"] = income["gesamtbetrag"]/income["anzahl_steuerpflichtige"]

In [ ]:
# select only Kreise
income_kreise = income[income["code"] != "DG"]                # have to do it separately, otherwise it doens't work
income_kreise = income_kreise[~income_kreise["code"].isin(land_codes)].reset_index(drop=True)            # ~ takes the opposite
income_kreise.head()

In [ ]:
# add state code
income_kreise["state_code"] = income_kreise["code"].apply(lambda x: x[:2])
income_kreise["state_code"].head()

In [ ]:
sorted_incomes = income_kreise.sort_values("year").reset_index(drop=True)

#### Maps income

In [ ]:
max(sorted_incomes["tax_perc"])

In [ ]:
min(sorted_incomes["tax_perc"])

In [ ]:
# fig = px.choropleth_map(
#     sorted_incomes[sorted_incomes["year"]==2021],
#     geojson=geojson,
#     locations="code",
#     featureidkey="properties.krs_code",
#     color="tax_perc",
#     hover_name="region",
#     zoom=5,
#     title="Income tax",
#     labels={'tax_perc': 'Income tax (%)'},
#     width=900, height=650,
#     range_color=(9, 28)
# )
# fig.update_layout(
#     map_center={"lat": 51, "lon": 10},
#     autosize=False,
#     margin={"r": 0, "t": 0, "l": 0, "b": 0}
# )
# fig.write_html("income_tax_2021.html")
# fig.show()

In [ ]:
# fig = px.choropleth_map(
#     sorted_incomes,
#     geojson=geojson,
#     locations="code",
#     featureidkey="properties.krs_code",
#     color="tax_perc",
#     animation_frame="year",
#     hover_name="region",
#     zoom=5,
#     title="Income tax",
#     labels={'tax_perc': 'Income tax (%)'},
#     width=900, height=650,
#     range_color=(9, 28)
# )
# fig.update_layout(
#     map_center={"lat": 51, "lon": 10},
#     autosize=False,
#     margin={"r": 0, "t": 0, "l": 0, "b": 0}
# )
# fig.write_html("income_tax_over_time.html")
# fig.show()

In [ ]:
min(sorted_incomes["income_per_capita"])

In [ ]:
max(sorted_incomes["income_per_capita"])

In [ ]:
# fig = px.choropleth_map(
#     sorted_incomes,
#     geojson=geojson,
#     locations="code",
#     featureidkey="properties.krs_code",
#     color="income_per_capita",
#     animation_frame="year",
#     hover_name="region",
#     zoom=5,
#     title="Income tax",
#     labels={'income_per_capita': 'Income per capita \n(TSD/year)'},
#     width=900, height=650,
#     range_color=(16, 82)
# )
# fig.update_layout(
#     map_center={"lat": 51, "lon": 10},
#     autosize=False,
#     margin={"r": 0, "t": 0, "l": 0, "b": 0}
# )
# fig.write_html("income_per_capita_over_time.html")
# fig.show()

<a id='maps'></a>
## [Combining maps](#P0)



In [ ]:
print(elections["election_year"].unique())
print(income["year"].unique())

In [ ]:
common_years = set(elections["election_year"].unique()) & set(income["year"].unique())
common_years

-> since we only have 4 years in which we have the data, we'll have to find a substitute (1-2 years before or 1-2 years after)

### Test with one year

In [ ]:
elections_winner_fig = px.choropleth_map(
    sorted_elects[sorted_elects["election_year"] == 2021],
    geojson=geojson,
    locations="county",
    featureidkey="properties.krs_code",
    color="winner",
    hover_name="winner",
    zoom=4.5,
    title="Elected party per district",     # not recognised?...
    labels={'winner': 'Winner party'},
    color_discrete_map={
        'cdu':'#003B6F',
        'spd':'#A6006B',
        'gruene':'#1AA037',
        'fdp':'#FFEF00',
        'linke_pds':'#E3000F',
        'afd':'#0489DB'
    },
    # width=900, height=650,
)
elections_winner_fig.update_layout(
    title_text="Elected party per district",
    map_center={"lat": 51, "lon": 10},
    autosize=False,
    margin={"r": 0, "t": 0, "l": 0, "b": 0}
)
elections_winner_fig.show()

In [ ]:
print(sorted_elects["far_left"].max())
print(sorted_elects["far_left_w_linke"].max())
print(sorted_elects["far_right"].max())

In [ ]:
sorted_elects["perc_far_left_w_linke"] = sorted_elects["far_left_w_linke"]*100
sorted_elects["perc_far_right"] = sorted_elects["far_right"]*100

In [ ]:
left_fig = px.choropleth_map(
    sorted_elects[sorted_elects["election_year"] == 2021],
    geojson=geojson,
    locations="county",
    featureidkey="properties.krs_code",
    color="perc_far_left_w_linke",
    hover_name="county",
    zoom=4.5,
    title="Percentage of people voting far left",
    labels={'perc_far_left_w_linke': 'People voting \nfar left \nand \ndie Linke (%)'},
    # width=900, height=650,
    range_color=(0, 100)
)
left_fig.update_layout(
    map_center={"lat": 51, "lon": 10},
    autosize=False,
    margin={"r": 0, "t": 0, "l": 0, "b": 0}
)
left_fig.show()

In [ ]:
right_fig = px.choropleth_map(
    sorted_elects[sorted_elects["election_year"] == 2021],
    geojson=geojson,
    locations="county",
    featureidkey="properties.krs_code",
    color="perc_far_right",
    hover_name="county",
    zoom=4.5,
    title="Percentage of people voting far left",
    labels={'perc_far_right': 'People voting \nfar right (%)'},
    # width=900, height=650,
    range_color=(0, 100)
)
right_fig.update_layout(
    map_center={"lat": 51, "lon": 10},
    autosize=False,
    margin={"r": 0, "t": 0, "l": 0, "b": 0}
)
right_fig.show()

In [ ]:
income_fig = px.choropleth_map(
    sorted_incomes[sorted_incomes["year"]==2021],
    geojson=geojson,
    locations="code",
    featureidkey="properties.krs_code",
    color="tax_perc",
    hover_name="region",
    zoom=4.5,
    title="Income tax",
    labels={'tax_perc': 'Income tax (%)'},
    # width=900, height=650,
    range_color=(9, 28)
)
income_fig.update_layout(
    map_center={"lat": 51, "lon": 10},
    autosize=False,
    margin={"r": 0, "t": 0, "l": 0, "b": 0}
)
income_fig.show()

Note: it is not possible to show choroplet_map side by side with subplots.      
The solution I found is the following

In [ ]:
from IPython.display import display, HTML

# Display them side by side using HTML flex
html_str = f"""
<div style="display: flex; flex-wrap: wrap; gap: 10px;">
    <div style="flex: 1 1 48%;">{elections_winner_fig.to_html(include_plotlyjs='cdn')}</div>
    <div style="flex: 1 1 48%;">{income_fig.to_html(include_plotlyjs=False)}</div>
    <div style="flex: 1 1 48%;">{left_fig.to_html(include_plotlyjs=False)}</div>
    <div style="flex: 1 1 48%;">{right_fig.to_html(include_plotlyjs=False)}</div>
</div>
"""

display(HTML(html_str))

<a id='CL'></a>
## [Conclusion](#P0)

This is a pre-written conclusion in which we have nice figures.

<div style="border-top:0.1cm solid #EF475B"></div>
    <strong><a href='#Q0'><div style="text-align: right"> <h3>End of this Notebook.</h3></div></a></strong>